In [1]:
import numpy as np
import torch 
from pathlib import Path

from gpmdm import GPMDM
from gpmdm_pf import GPMDM_PF
from dataset_utils.mocap_labels import WALK_TRIALS_TRAIN, WALK_TRIALS_TEST, RUN_TRIALS_TRAIN, RUN_TRIALS_TEST
import dataset_utils.select_joints as select_joints

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
config_path = Path.cwd() / 'models' / 'gpmdm' / 'gpmdm_config.pth'
state_path = Path.cwd() / 'models' / 'gpmdm' / 'gpmdm_state.pth'

In [3]:
reduced_columns = WALK_TRIALS_TEST[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [4]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")

Number of DOFs: 35


In [5]:
d = 3 # latent space dimension
DOFs = len(reduced_columns)  # number of degrees of freedom

dyn_back_step = 1 # Number of time steps to look back in the dynamics GP

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(dyn_back_step*d)  # Lengthscales for latent dynamics GP
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(dyn_back_step*d + 1)  # Linear coefficients for latent dynamics GP


In [6]:
gpdm = GPMDM(
        D=DOFs,
        d=d,
        n_classes=2,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )

In [7]:
gpdm.load(config_path, state_path)


GPDM correctly loaded


In [8]:
# Markov Transition Matrix between classes
markov_transition_matrix = torch.tensor(
    [[0.9, 0.1], 
     [0.1, 0.9]])

# Build a gpdm_pf model
gpdm_pf = GPMDM_PF(gpdm, 
                   markov_switching_model=markov_transition_matrix,
                   num_particles=100,
                   n_fresh_particles_at_each_timestep=99)
   

In [ ]:
for i, trial in enumerate(TEST_TRIALS):
    label = TRUTH_LABELS[i]

    # Get the data in an array format
    data_arr = trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

    gpdm_pf.reset()
    correct = 0
    total = 0

    # Run the filter bank on the trial
    for data in data_arr:
        gpdm_pf.update(data)
        predicted_label = gpdm_pf.get_most_likely_class()
        latest_lls = gpdm_pf.class_probabilities()
        walk_ll = latest_lls[0]
        run_ll = latest_lls[1]

        if predicted_label == label:
            correct += 1
        
        total += 1

    accuracy = correct / total

    print(f"Trial {i}, Accuracy: {accuracy}")
    print(f"Trial {i} Final, True label: {label}, Predicted label: {predicted_label}, Walk LL: {walk_ll}, Run LL: {run_ll}")


Trial 0, Accuracy: 1.0
Trial 0 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 1, Accuracy: 1.0
Trial 1 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 2, Accuracy: 1.0
Trial 2 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 3, Accuracy: 1.0
Trial 3 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 4, Accuracy: 1.0
Trial 4 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 5, Accuracy: 1.0
Trial 5 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 6, Accuracy: 0.9393939393939394
Trial 6 Final, True label: 1, Predicted label: 0, Walk LL: 1.0, Run LL: 1.8266115766635082e-39
Trial 7, Accuracy: 0.7368421052631579
Trial 7 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 8, Accuracy: 0.7777777777777778
Trial 8 Final, True label: 1, Predicted label: 1, Walk LL: 0.0, Run LL: 1.0
Trial 9, Accuracy: 0.9285714285714286
Trial 9